### Test Capser-suggested fixes on fin datasets that fail with alphashape

In [5]:
import numpy as np
import scipy

from sklearn import svm
from sklearn.linear_model import SGDOneClassSVM
from skimage import measure

# remeshing
import pyvista
import pyacvd
import pymeshfix
import igl

# plotting
import meshplot

# NL additions
import pandas as pd
import os
import glob2 as glob
import trimesh

In [6]:
# get list of refined fin mesh objects
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"

fin_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*smoothed_fin_mesh*")))
raw_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*rawfin_mesh*")))
fin_df_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*upsampled*")))
fin_df_c_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*data.csv")))

In [7]:
file_ind = 3

fin_mesh =trimesh.load(fin_mesh_list[file_ind])
raw_fin_mesh =trimesh.load(raw_mesh_list[file_ind])
fin_df = pd.read_csv(fin_df_list[file_ind])
fin_df_c = pd.read_csv(fin_df_c_list[file_ind])

### Register points to a regular 3D grid 

In [8]:
xyz_raw = fin_df[["XB", "YB", "ZB"]].to_numpy()

# mm = np.min(xyz_raw)
# xyz = xyz_raw - mm
# mx = np.max(xyz)
# xyz = xyz / mx
xyz = xyz_raw.copy()

f_max = np.max(xyz, axis=0)
f_min = np.min(xyz, axis=0)

In [9]:
res = np.ceil((f_max - f_min) / 1).astype(int)
print(res)

# Buid grid to evaluate SDF
x = np.linspace(f_min[0], f_max[0], res[0])
y = np.linspace(f_min[1], f_max[1], res[1])
z = np.linspace(f_min[2], f_max[2], res[2])
x_grid, y_grid, z_grid = np.meshgrid(x, y, z)#, indexing='ij')

[113  99  55]


In [10]:
from scipy.spatial import KDTree

# set NN grid points to 1, else 0
xyz_array = np.c_[x_grid.ravel(), y_grid.ravel(), z_grid.ravel()]

# Find the nearest points on the fin boundary for each ellipsoid boundary point
tree = KDTree(xyz)
distances, indices = tree.query(xyz_array)
distances.shape

(615285,)

In [11]:
# p_vec = np.ones((xyz_array.shape[0],))
d_grid =  np.reshape(distances, x_grid.shape)

In [12]:
np.percentile(distances, 90)

np.float64(19.085209309430976)

In [13]:
# Marching Cubes magic!
verts, faces, _, _ = measure.marching_cubes(d_grid, level = 3, spacing = np.asarray([1.0, 1.0, 1.0]))

In [14]:
verts.shape

(55843, 3)

In [65]:
# marching cubes guarantees topologically correct results (very useful)
# but that doesn't mean the mesh is going to look pretty!
meshplot.plot(verts, faces, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.0, 56.…

In [67]:
# util functions for converting to and from Pyvista mesh format
def mesh_to_pyvista(v, f):
    n, m = f.shape
    threes = np.full((n, 1), 3)
    face_arr = np.hstack((threes, f)).flatten()
    return pyvista.PolyData(v, face_arr)


def pyvista_to_mesh(mesh):
    v = mesh.points
    f = mesh.faces.reshape(-1, 4)[:, 1:4]
    return v, f


In [70]:
# this can give a depreciation warning but it is fine
mesh = mesh_to_pyvista(verts, faces)

# target mesh resolution
target_verts = 2500

clus = pyacvd.Clustering(mesh)
clus.subdivide(2)
clus.cluster(target_verts)

remesh = clus.create_mesh()

v2, f2 = pyvista_to_mesh(remesh)

# pymeshfix is often necessary here to get rid of non-manifold vertices
v2, f2 = pymeshfix.clean_from_arrays(v2, f2)

In [71]:
p = meshplot.plot(v2, f2, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(48.962877…

In [ ]:
vo, fo = fin_mesh.vertices, raw_fin_mesh.faces

# this can give a depreciation warning but it is fine
mesh = mesh_to_pyvista(vo, fo)

# target mesh resolution
target_verts = 2500

clus = pyacvd.Clustering(mesh)
clus.subdivide(2)
clus.cluster(target_verts)

remesh = clus.create_mesh()

v2, f2 = pyvista_to_mesh(remesh)

# pymeshfix is often necessary here to get rid of non-manifold vertices
v2, f2 = pymeshfix.clean_from_arrays(v2, f2)

In [ ]:
p = meshplot.plot(v2, f2, shading={"wireframe": True})

In [76]:
mesh = trimesh.Trimesh(vertices=v2, faces=f2)
mesh.is_watertight

True

In [77]:
mesh.vertices.shape

(2488, 3)

In [16]:
import alphashape 

test = alphashape.alphashape(verts, alpha=20)

In [19]:
fin_mesh.vertices.shape

(2499, 3)